In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.10.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: AWSSDK.BedrockRuntime, 3.7.302.7"
#r "nuget: Rockhead.Extensions, 0.1.2-preview"

In [ ]:
using System;
using System.IO;
using System.Threading;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Amazon;
using Amazon.Util;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Rockhead.Extensions; // TODO: consider contributing
using Rockhead.Extensions.Meta;

public class SemanticBedrock : Microsoft.SemanticKernel.TextGeneration.ITextGenerationService
{
	public IReadOnlyDictionary<string, object?> Attributes => new Dictionary<string, object?>();

	public SemanticBedrock()
	{
		Console.WriteLine("SemanticBedrock constructor");
	} // TODO ctor: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/Connectors/Connectors.Google/Services/GoogleAIGeminiChatCompletionService.cs#L31

	public IAsyncEnumerable<StreamingTextContent> GetStreamingTextContentsAsync(string prompt, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		Console.WriteLine("GetStreamingTextContentsAsync");
		throw new NotImplementedException();
	}

	public Task<IReadOnlyList<TextContent>> GetTextContentsAsync(string prompt, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		// var result = new List<TextContent> { new("Hello World!") };
		Console.WriteLine("GetTextContentsAsync");
		// return Task.FromResult<IReadOnlyList<TextContent>>(result);

		var builder = new ConfigurationBuilder()
			.SetBasePath(Directory.GetCurrentDirectory())
			.AddJsonFile(Path.Combine(
				Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
				"Microsoft",
				"UserSecrets",
				"2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
				"secrets.json")
			);

		IConfigurationRoot configuration = builder.Build();

		string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
		string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

		if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
		{
			Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
			throw new Exception("AWSAccessKeyId or AWSSecretAccessKey is not set");
		}


		string modelId = "meta.llama3-70b-instruct-v1:0";
		AmazonBedrockRuntimeClient awsClient = new(awsAccessKeyId, awsSecretAccessKey, RegionEndpoint.USEast1);
		string payload = new JsonObject()
		{
			{ "prompt", "Test prompt" },
			{ "max_gen_len", 512 },
			{ "temperature", 0.5 }
		}.ToJsonString();

		try
		{
			var response = awsClient.InvokeModelAsync(new InvokeModelRequest
			{
				ModelId = modelId,
				Body = AWSSDKUtils.GenerateMemoryStreamFromString(payload),
				ContentType = "application/json",
				Accept = "application/json"
			}).GetAwaiter().GetResult();
			Console.WriteLine(response);
			Console.WriteLine(response?.HttpStatusCode);
			Console.WriteLine(response?.Body?.ToString());

			// use system.text.json to parse the response into testResponse object
			var jsonString = (new StreamReader(response.Body).ReadToEndAsync()).GetAwaiter().GetResult();
			Console.WriteLine(jsonString);

			var testResponse = JsonSerializer.Deserialize<Llama2Response>(jsonString) ?? new Llama2Response();
			Console.WriteLine(testResponse);
			Console.WriteLine(testResponse.Generation);
			Console.WriteLine(testResponse.PromptTokenCount);
			Console.WriteLine(testResponse.GenerationTokenCount);
			Console.WriteLine(testResponse.StopReason);

			return Task.FromResult<IReadOnlyList<TextContent>>(new List<TextContent> { new(testResponse.Generation) });
		}
		catch (Exception ex)
		{
			Console.WriteLine($"Error invoking model: {ex.Message}");
			return Task.FromResult<IReadOnlyList<TextContent>>(new List<TextContent> { new("Error invoking model") });
		}
	}
}

# Load OpenAI key from dotnet User-Secrets and setup Semantic Kernel

In [ ]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextGeneration;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();
string openAIAPIKey = configuration["OpenAIAPIKey"];
string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

Console.WriteLine("OpenAIAPIKey is set");

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.Services.AddSingleton<ITextGenerationService, SemanticBedrock>();
kernelBuilder.Services.AddSingleton<ITextGenerationService, SemanticBedrock>((_) => new SemanticBedrock());
// kerbelBuilder.AddOpenAITextEmbeddingGeneration("text-embedding-3-small", openAIAPIKey);
var kernel = kernelBuilder.Build();

// var embeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

const string functionDefinition = "Write one paragraph on {{$input}}";
var paragraphWritingFunction = kernel.CreateFunctionFromPrompt(functionDefinition);

const string Input = "Why AI is awesome";
Console.WriteLine($"Function input: {Input}\n");
var result = await paragraphWritingFunction.InvokeAsync(kernel, new() { ["input"] = Input });
Console.WriteLine($"Function output: {result}");
// kernel.InvokePromptAsync("Write one paragraph on Why AI is awesome", cancellationToken: default).Wait();

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

// var executionSettings = new OpenAIPromptExecutionSettings  //TODO bedrock settings
// {
//     MaxTokens = 2000,
//     Temperature = 0.7,
//     TopP = 0.5
// };
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt);//, executionSettings);
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

In [ ]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);

arguments["userInput"] = "Alright, but I'm looking for something in the fantasy genre";

var answer = await chatFunction.InvokeAsync(kernel, arguments);

var result = $"\nUser: {arguments["userInput"]}\nAI: {answer}\n";
history += result;

arguments["history"] = history;

// Show the response
Console.WriteLine(result);

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.10.0-alpha"
#r "nuget: System.Interactive.Async, 6.0.1"

using System.Linq;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0050
var memoryBuilder = new MemoryBuilder();

//TODO replace with Bedrock
memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", openAIAPIKey);

memoryBuilder.WithMemoryStore(new VolatileMemoryStore()); // TODO try some Bedrock memory store
var memory = memoryBuilder.Build();

const string MemoryCollectionName = "aboutMe";
await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");

var response = await memory.SearchAsync(MemoryCollectionName, "What is my name?").FirstOrDefaultAsync();
Console.WriteLine(response);
Console.WriteLine(response?.Display());

In [ ]:
#r "nuget: Qdrant.Client, 1.9.0"

In [ ]:
docker start quokkavec-qdrant

# Setup Qdrant vector DB, setup Collection and fill it with embeddings

In [ ]:
var pathToIndex = "c:/temp/quokkavec-test";

In [ ]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var collectionName = "test";
var qdrantClient = new QdrantClient("localhost");
if (!await qdrantClient.CollectionExistsAsync(collectionName))
{
	await qdrantClient.CreateCollectionAsync(collectionName, new VectorParams { Size = 1536, Distance = Distance.Cosine });
}

// index all files in the folder
var files = Directory.GetFiles(pathToIndex, "*", SearchOption.AllDirectories);
int i = 0;
foreach (var file in files)
{
	var text = await File.ReadAllTextAsync(file);
	// take only first 1k characters
	text = text.Substring(0, Math.Min(1000, text.Length));
	ReadOnlyMemory<float> embedding = await embeddingGenerationService.GenerateEmbeddingAsync(text);

	var points = new List<PointStruct>()
	{
		new PointStruct
		{
			Id = new PointId { Num = (ulong)i },
			Vectors = embedding.ToArray(),
			Payload = { { "fileName", file } }
		}
	};
	var updateResult = await qdrantClient.UpsertAsync(collectionName, points);
	Console.WriteLine($"File {file} indexed, result: {updateResult}");

	i++;
}

In [ ]:
string searchQuery = "Latest news";

In [ ]:
#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var queryVector = await embeddingGenerationService.GenerateEmbeddingAsync(searchQuery);
var searchResult = await qdrantClient.SearchAsync(collectionName, queryVector.ToArray(), limit: 2);

foreach (var result in searchResult)
{
	Console.WriteLine($"File: {result.Payload["fileName"]}, Score: {result.Score}");
}